In [1]:
import sys
sys.path.append('..')
import numpy as np
import pandas as pd
import ast
from src.model import GoldenRetriever
from src.utils import read_txt, split_txt, make_pred, make_iscorr, make_closewrong, make_finetune, make_contrastive_finetune

/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)
/anaconda/envs/py35/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
kb_location='../data/fund_guide_tnc_full.txt'
text = split_txt(read_txt(kb_location))
print(len(text))

64


In [3]:
gr=GoldenRetriever(lr=0.3)

Instructions for updating:
Colocations handled automatically by placer.


W1108 09:33:00.163685 140367382079232 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/control_flow_ops.py:3632: colocate_with (from tensorflow.python.framework.ops) is deprecated and will be removed in a future version.
Instructions for updating:
Colocations handled automatically by placer.


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


I1108 09:33:14.989733 140367382079232 saver.py:1483] Saver not created because there are no variables in the graph to restore


INFO:tensorflow:Saver not created because there are no variables in the graph to restore


...

Instructions for updating:
Use tf.cast instead.


W1108 09:33:29.671785 140367382079232 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/losses/losses_impl.py:331: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


Instructions for updating:
Use tf.cast instead.


W1108 09:33:29.810614 140367382079232 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/ops/math_ops.py:3066: to_int32 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.cast instead.


model initiated!


In [4]:
gr.load_kb(kb_location)
print(len(gr.text))

knowledge base lock and loaded!
1


In [5]:
import pandas as pd
# df_queries = pd.read_excel('../data/test_dataset_clean.xls', encoding='iso-8859-1')
df_queries = pd.read_csv('../data/labeled_dataset_v2.csv', encoding='iso-8859-1')
print(len(df_queries))
df_queries.dropna(axis=0, subset=['answer'], inplace=True)
df_queries['answer']=df_queries.apply(lambda x: ast.literal_eval(x['answer']), axis=1)
print(len(df_queries))

113
105


In [6]:
ind = np.arange(len(df_queries))
np.random.seed(1)
np.random.shuffle(ind)
train_ind=ind[:80]
test_ind=ind[80:]

In [7]:
%%time
for k in range(3):
    df_queries['predictions']=df_queries.apply(lambda x: make_pred(x, gr, top_k=k+1), axis=1)
    df_queries['iscorr']=df_queries.apply(lambda x: make_iscorr(x, 'predictions'), axis=1)
    print(df_queries.iloc[test_ind]['iscorr'].sum()/len(df_queries.iloc[test_ind]))

0.28
0.44
0.48
CPU times: user 16.5 s, sys: 8.29 s, total: 24.8 s
Wall time: 6.54 s


## use trained model

In [12]:
# gr.restore('/data/home/lik/info_retrieve/google_use_qa_tuned/variables-0')
gr.restore('/data/home/lik/info_retrieve/google_use_nrf_tuned/variables-0')
gr.load_kb(kb_location)

Instructions for updating:
Use standard file APIs to check for files with this prefix.


W1108 09:23:00.819271 139950127408896 deprecation.py:323] From /anaconda/envs/py35/lib/python3.5/site-packages/tensorflow/python/training/saver.py:1266: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


INFO:tensorflow:Restoring parameters from /data/home/lik/info_retrieve/google_use_nrf_tuned/variables-0


I1108 09:23:00.821304 139950127408896 saver.py:1270] Restoring parameters from /data/home/lik/info_retrieve/google_use_nrf_tuned/variables-0


model checkpoint restored!
knowledge base lock and loaded!


In [10]:
%%time
for k in range(5):
    df_queries['predictions']=df_queries.apply(lambda x: make_pred(x, gr, top_k=k+1), axis=1)
    df_queries['iscorr']=df_queries.apply(lambda x: make_iscorr(x, 'predictions'), axis=1)
    print(df_queries.iloc[test_ind]['iscorr'].sum()/len(df_queries.iloc[test_ind]))

0.32
0.36
0.44
0.48
0.68
CPU times: user 23.6 s, sys: 13.8 s, total: 37.4 s
Wall time: 6.45 s


## Finetuning

In [10]:
# df_queries['closewrong']=df_queries.apply(lambda x: make_closewrong(x), axis=1)

In [11]:
for _ in range(5):
    df_queries['closewrong']=df_queries.apply(lambda x: make_closewrong(x), axis=1)
    df_queries.iloc[train_ind].apply(lambda x: make_finetune(x, gr), axis=1)
    gr.load_kb('../data/fund_guide_tnc_full.txt')
    df_queries['predictions_finetune']=df_queries.apply(lambda x: make_pred(x, gr), axis=1)
    df_queries['iscorr_finetune']=df_queries.apply(lambda x: make_iscorr(x, 'predictions_finetune'), axis=1)
    print(df_queries.iloc[test_ind]['iscorr_finetune'].sum()/len(df_queries.iloc[test_ind]))

0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.54717773
0.0
0.0
0.041044533
0.0
0.0
0.048087776
0.0
0.017304718
0.010251403
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.02643317
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.08591068
0.0
0.0
0.0
0.0
0.10307461
0.0
0.0
0.0
0.0
0.0
0.0
0.04044366
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.027150929
0.0
knowledge base lock and loaded!
0.6
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.5076647
0.0
0.0
0.0
0.0
0.031972647
0.008863747
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.009117842
0.0
0.0075131655
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
knowledge base lock and loaded!
0.48
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.59946454
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.

In [ ]:
%%time
gr.load_kb('../data/fund_guide_tnc_full.txt')
for k in range(5):
    df_queries['predictions_finetune']=df_queries.apply(lambda x: make_pred(x, gr, top_k=k+1), axis=1)
    df_queries['iscorr_finetune']=df_queries.apply(lambda x: make_iscorr(x, 'predictions_finetune'), axis=1)
    print(df_queries.iloc[test_ind]['iscorr_finetune'].sum()/len(df_queries.iloc[test_ind]))

knowledge base lock and loaded!
0.36
0.44


In [23]:
df_queries

,queries,Guidelines for the Management of Competitive RnD Grants (1 July 2016),Terms and Conditions for Competitive Grants (1 July 2016),answer,predictions,iscorr,closewrong,predictions_finetune,iscorr_finetune
0,SMRT-NTU Corp Lab would like to get NRF approv...,NaN,15. Publications of Results and Findings,[12],"[12, 46, 33]",1,33,"[12, 46, 8]",1
1,We would like to request for a nationality wai...,Nationality Waiver,NaN,[0],"[0, 1, 10]",1,10,"[1, 8, 0]",1
2,We would like to hire Mr Mao Fubing as a Resea...,Nationality Waiver,NaN,[0],"[0, 18, 33]",1,18,"[0, 1, 8]",1
3,For getting approval from NRF to hire non Sing...,Nationality Waiver,NaN,[0],"[0, 1, 17]",1,9,"[0, 1, 8]",1
4,"During the SC meeting in Sept 18, it was agree...",EOM,NaN,[1],"[32, 18, 19]",0,6,"[1, 3, 8]",1
5,I would like to seek your opinion on this matt...,Overseas Travel Related Expenses,NaN,[2],"[2, 17, 11]",1,3,"[2, 1, 0]",1
6,"As per our conversation held this pm, you upda...",Nationality Waiver,NaN,[0],"[0, 1, 17]",1,1,"[0, 8, 1]",1
7,"If a research project group , has 2 Research F...",EOM,NaN,[1],"[1, 37, 18]",1,37,"[1, 0, 2]",1
8,"As per conversation previously , it was mentio...",EOM,NaN,[1],"[1, 0, 17]",1,6,"[0, 1, 3]",1
9,"As mentioned in the emailÃâs subject, I wou...",Equipment and Other Operating Expenses (OOE),NaN,[3],"[3, 8, 43]",1,14,"[3, 8, 1]",1


In [14]:
# gr.export(savepath='../google_use_nrf_tuned/variables')


In [10]:
gr.make_query('How can I apply for an extension?', top_k=3)

(['Grant Extension\n27. Request for grant extension should be made before the last 6 months of the original end of the Term. The PI must ensure sufficient funds in each vote to support the extension request. Any variation requests necessary to meet the extension period must be made known as part of the extension request.\n28. A one-off project extension should not be more than a total of 6 months. An extension beyond 6 months will require compelling justification. No additional funds should be given for any extensions.\n',
  '19. Completion/Extension\n19.1 Unless earlier terminated in accordance with this Contract or if Grantor agrees in writing to an extension of time, this Contract shall end upon the expiry of the Term. Unless otherwise specifically provided in the Letter of Award, any application for extension of time shall be made to Grantor no later than six (6) months before the original end of the Term unless there is compelling justification for submission of a late application